<a href="https://colab.research.google.com/github/karank85/speech-recognition/blob/main/Copy_of_Project_2_Non_DL_Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import numpy as np
from numpy import ndarray
import pandas as pd

import librosa

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa.display
from tqdm import tqdm


import glob
import torch

In [2]:
librosa.__version__

'0.10.1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Assumptions:
# - The transcription file is located in the same directory as the audio files.
class AudioDataset:
  """
  Class for loading and storing audio data.
  """

  def __init__(self):
    self.df = pd.DataFrame(columns=['id', 'path', 'transcription'])

  def load_transcriptions(self, directory_path: str) -> bool:
    """
    Load all transcriptions from a given directory, including subdirectories.
    Returns False if no transcription files were found, or if any failed to load.
    """
    transcriptions_path = glob.glob(
        f"{directory_path}/**/*.trans.txt",
        recursive=True
    )

    if len(transcriptions_path) == 0:
      return False

    for path in transcriptions_path:
      if not self.load_transcription_file(path):
        return False

    return True



  def load_transcription_file(self, file_path: str) -> bool:
    """
    Parse transcription file and records the audio ID - subtitle mapping.
    Returns False if the file could not be read.
    """
    with open(file_path, "r") as file:
      file_directory = os.path.dirname(file_path)

      lines = file.read().split("\n")
      for line in lines:
        if len(line.strip()) == 0:
          continue
        splitter = line.split(" ")
        file_name = splitter[0]
        file_content = ' '.join(splitter[1:])
        self.df.loc[len(self.df)] = {
            'id':file_name,
            'transcription':file_content,
            'path': f'{file_directory}/{file_name}.flac'
        }
      return True
    return False

  def keys(self):
    return iter(self.df['id'])

  def get(self, id: int):
    """
    Retrieve a dataframe row from ID.
    """
    return self.df.loc[self.df['id'] == id]

In [ ]:
ds = AudioDataset()

In [ ]:
ds.load_transcriptions("/content/drive/MyDrive/test_run_dataset")

### Extracting Features

In [ ]:
from librosa.feature import mfcc
import librosa
audio, sampling_freq = librosa.load(ds.df.head()['path'].values[0])
mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
print('\nNumber of windows =', mfcc_features.shape[0])
print('Length of each feature =', mfcc_features.shape[1])

In [ ]:
mfcc_features = mfcc_features.T
plt.matshow(mfcc_features)
plt.title('MFCC')

In [ ]:
!pip install hmmlearn
!pip install features

In [ ]:
import os
import numpy as np
from scipy.io import wavfile
from hmmlearn import hmm #importing GaussianHMM
import librosa # reading wavefilesfrom librosa.feature import mfcc #to extract mfcc features

In [ ]:
class HMMTrainer(object):
  def __init__(self, model_name='GaussianHMM', n_components=4):
     self.model_name = model_name
     self.n_components = n_components

     self.models = []
     if self.model_name == 'GaussianHMM':
        self.model=hmm.GaussianHMM(n_components=4)
     else:
        print("Please choose GaussianHMM")

  def train(self, X):
      self.models.append(self.model.fit(X))

  def get_score(self, input_data):
      return self.model.score(input_data)

In [ ]:
ds.df

In [ ]:
hmm_models = []
X = np.array([])
y_words = []
label = []

for index, row in ds.df.iloc[:-1, :].iterrows():
   # Read the input file
   audio, sampling_freq = librosa.load(row['path'])
   # Extract MFCC features
   mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
   # Append to the variable X
   if len(X) == 0:
     X = mfcc_features[:, :15]  # 15 here denotes the number of MFCC coefficients to consider
   else:
     X = np.append(X, mfcc_features[:, :15], axis=0)
   # Append the label
   y_words.append(row['transcription'])
  #  print('X.shape =', X.shape)

   # Train HMM model for this iteration
   hmm_trainer = HMMTrainer()
   hmm_trainer.train(X)  # Train using the current MFCC features
   hmm_models.append((hmm_trainer, row['transcription']))


In [ ]:
test = ds.df.iloc[-1,:]
# Read the input file
audio, sampling_freq = librosa.load(test['path'])
# Extract MFCC features
mfcc_features = librosa.feature.mfcc(sr=sampling_freq, y=audio)
X = mfcc_features[:, :15]
y_word = test['transcription']

In [ ]:
scores = []
for hmm_model, _ in hmm_models:
    score = hmm_model.get_score(X)
    scores.append(score)

print(scores)
index = np.array(scores).argmax()
# Print the output
print("\nTrue:", y_word)

print("Predicted:", hmm_models[index][1])